In [1]:
# Benchmark: Flow around a cylinder

In [2]:
from ngsolve import *
from ngsolve.webgui import Draw

In [3]:
from netgen.occ import *

shape = Rectangle(2,0.41).Circle(0.2,0.2,0.05).Reverse().Face()
shape.edges.name="wall"
shape.edges.Min(X).name="inlet"
shape.edges.Max(X).name="outlet"
Draw (shape);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [4]:
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.07)).Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [5]:
V = VectorH1(mesh,order=3, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=2)
X = V*Q

u,p = X.TrialFunction()
v,q = X.TestFunction()

nu = 0.001  # viscosity
stokes = (nu*InnerProduct(grad(u), grad(v))+ \
    div(u)*q+div(v)*p - 1e-10*p*q)*dx

a = BilinearForm(stokes).Assemble()

# nothing here ...
f = LinearForm(X).Assemble()

# gridfunction for the solution
gfu = GridFunction(X)

In [6]:
umean = 0.3
uin = CoefficientFunction( (umean*4*y*(0.41-y)/(0.41*0.41), 0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

Draw (gfu.components[0], mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [7]:
tau = 0.001 # timestep

mstar = BilinearForm(u*v*dx+tau*stokes).Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

In [8]:
conv = BilinearForm(X, nonassemble=True)
conv += (Grad(u) * u) * v * dx

In [9]:
t = 0; i = 0
tend = 5
vel = gfu.components[0]
scene = Draw (gfu.components[0], mesh, min=0, max=0.4, autoscale=False)

with TaskManager():
    while t < tend:
        res = conv.Apply(gfu.vec) + a.mat*gfu.vec
        gfu.vec.data -= tau * inv * res    

        t = t + tau; i = i + 1
        if i%10 == 0: 
            scene.Redraw()
            # print(f"t = {t}", end='\r')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [10]:
### Test case 2D-2 (unsteady):

In [ ]:
V = VectorH1(mesh,order=3, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=2)
X = V*Q

u,p = X.TrialFunction()
v,q = X.TestFunction()

nu = 0.001 
stokes = (nu*InnerProduct(grad(u), grad(v))+ \
    div(u)*q+div(v)*p - 1e-10*p*q)*dx

a = BilinearForm(stokes).Assemble()
f = LinearForm(X).Assemble()
gfu = GridFunction(X)

umean = 1.5 # New value for 2D-2 [cite: 47]
uin = CoefficientFunction( (umean*4*y*(0.41-y)/(0.41*0.41), 0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

inv_stokes = a.mat.Inverse(X.FreeDofs())
res = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res
# Draw (gfu.components[0], mesh); # Visualize initial state if desired

# --- Time Stepping Setup ---
tau = 0.001 # timestep (may need to be smaller for Re=100)

mstar = BilinearForm(u*v*dx+tau*stokes).Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")
conv = BilinearForm(X, nonassemble=True)
conv += (Grad(u) * u) * v * dx

t = 0; i = 0
# tend = 5 # Original end time
tend = 20 # Increased end time for 2D-2 (adjust as needed)

vel = gfu.components[0]
# scene = Draw (gfu.components[0], mesh, min=0, max=0.4, autoscale=False) # Original draw
scene = Draw (gfu.components[0], mesh, min=0, max=1.8, autoscale=False) # Adjusted max value

# --- Add lists to store results for post-processing ---
results_t = []
results_cl = [] # Need to compute cL
results_cd = [] # Need to compute cD

with TaskManager():
    while t < tend:
        # --- Apply boundary conditions (constant for 2D-2) ---
        # gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet")) # Already set initially

        res = conv.Apply(gfu.vec) + a.mat*gfu.vec
        gfu.vec.data -= tau * inv * res

        t = t + tau; i = i + 1

        # --- Calculate and store cL, cD here (Requires defining integral forms) ---
        # Placeholder:
        current_cl = 0.0 # Replace with actual calculation
        current_cd = 0.0 # Replace with actual calculation
        results_t.append(t)
        results_cl.append(current_cl)
        results_cd.append(current_cd)
        # --- End calculation ---

        if i%50 == 0: # Draw less frequently for longer simulation
            scene.Redraw()
            # print(f"t = {t:.3f}", end='\r')

# --- Post-processing code needed here to analyze results_t, results_cl, results_cd ---
# --- e.g., find period, max values, Strouhal number ---